In [4]:
cd /Users/cfe/Dev/fastapi-nosql/

/Users/cfe/Dev/fastapi-nosql


In [22]:
from requests_html import HTML
from slugify import slugify
import pprint
import re
from app import scraper

In [6]:
url = "https://www.amazon.com/Vitamix-E310-Explorian-Professional-Grade-Container/dp/B0758JHZM3/ref=sr_1_3?dchild=1&keywords=vitamix&qid=1632337952&sr=8-3&th=1"

In [7]:
s = scraper.Scraper(url=url, endless_scroll=True)

In [8]:
html_str = s.get()

In [9]:
html_obj = HTML(html=html_str)

In [16]:
def extract_element_text(html_obj, element_id):
    el =  html_obj.find(element_id, first=True)
    if not el:
        return ''
    return el.text

In [17]:
price_str = extract_element_text(html_obj, '#priceblock_ourprice')
price_str

'$326.32'

In [18]:
title_str = extract_element_text(html_obj, "#productTitle")
title_str

'Vitamix E310 Explorian Blender, Professional-Grade, 48 oz. Container, Black'

In [19]:
def extract_tables(html_obj):
    return html_obj.find("table")

In [21]:
tables = extract_tables(html_obj)
tables

[<Element 'table' class=('a-lineitem',)>,
 <Element 'table' class=('a-normal', 'a-spacing-micro')>,
 <Element 'table' id='HLCXComparisonTable' class=('a-bordered', 'a-horizontal-stripes', 'a-spacing-none', 'a-size-base', 'comparison_table')>,
 <Element 'table' id='productDetails_detailBullets_sections1' class=('a-keyvalue', 'prodDetTable') role='presentation'>,
 <Element 'table' id='productDetails_feedback_sections' class=('a-keyvalue', 'prodDetTable') role='presentation'>,
 <Element 'table' id='histogramTable' class=('a-normal', 'a-spacing-none') role='presentation'>,
 <Element 'table' id='histogramTable' class=('a-normal', 'a-align-center', 'a-spacing-base')>,
 <Element 'table' class=('navFooterMoreOnAmazon',) cellspacing='0'>]

In [26]:
def extract_price_from_string(value: str, regex=r"[\$]{1}[\d,]+\.?\d{0,2}"):
    x = re.findall(regex, value)
    val = None
    if len(x) == 1:
        val = x[0]
    return val

In [27]:
def extract_dataset(tables):
    dataset = {}
    for table in tables:
        for tbody in table.element.getchildren():
            for tr in tbody.getchildren():
                row = []
                for col in tr.getchildren():
                    content = ""
                    try:
                        content = col.text_content()
                    except:
                        pass
                    if content != "":
                        _content = content.strip()
                        row.append(_content)
                if len(row) != 2:
                    continue
                key = row[0]
                value = row[1]
                
                # print(key, value)
                data = {}
                key = slugify(key)
                if key in dataset:
                    continue
                else:
                    if "$" in value:
                        new_key = key
                        old_key = f'{key}_raw'
                        new_value = extract_price_from_string(value)
                        old_value = value
                        dataset[new_key] = new_value
                        dataset[old_key] = old_value
                    else:
                        dataset[key] = value
    return dataset


In [28]:
dataset = extract_dataset(tables)
pprint.pprint(dataset)

{'asin': 'B0758JHZM3',
 'best-sellers-rank': '#3,172 in Kitchen & Dining (See Top 100 in Kitchen & '
                      'Dining)\n'
                      '\n'
                      '#8 in Blender Replacement Parts\n'
                      '\n'
                      '#9 in Cocktail Drinkware\n'
                      '\n'
                      '#22 in Countertop Blenders',
 'brand': 'Vitamix',
 'color': 'Black',
 'country-of-origin': 'USA',
 'customer-reviews': '/*\n'
                     '* Fix for UDP-1061. Average customer reviews has a small '
                     'extra line on hover\n'
                     '* '
                     'https://omni-grok.amazon.com/xref/src/appgroup/websiteTemplates/retail/SoftlinesDetailPageAssets/udp-intl-lock/src/legacy.css?indexName=WebsiteTemplates#40\n'
                     '*/\n'
                     '.noUnderline a:hover {\n'
                     'text-decoration: none;\n'
                     '}\n'
                     '\n'
                